In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
from time import time
import xgboost
from matplotlib import pyplot as plt

from xgboost import XGBRegressor
from sklearn.preprocessing import RobustScaler, Imputer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn import metrics
from datetime import datetime, time

In [10]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.shape

(1460, 81)

In [11]:
test.shape

(1459, 80)

In [12]:
test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [13]:
train = train.dropna(axis=1, thresh=1168)
test = test.dropna(axis=1, thresh=1168)
len(test)

1459

In [14]:
df_obj = train.select_dtypes(include=[np.object])
df_obj2 = test.select_dtypes(include=[np.object])

In [15]:
df_obj_dums = pd.get_dummies(df_obj, drop_first=True)
df_obj_dums2 = pd.get_dummies(df_obj2, drop_first=True)
len(test)

1459

In [16]:
drops = ['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','SaleType','SaleCondition']

In [17]:
train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [18]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [19]:
train = pd.concat([train, df_obj_dums], axis=1)
test = pd.concat([test, df_obj_dums2], axis=1)
test.columns

Index([u'Id', u'MSSubClass', u'LotFrontage', u'LotArea', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'MasVnrArea',
       u'BsmtFinSF1',
       ...
       u'SaleType_ConLI', u'SaleType_ConLw', u'SaleType_New', u'SaleType_Oth',
       u'SaleType_WD', u'SaleCondition_AdjLand', u'SaleCondition_Alloca',
       u'SaleCondition_Family', u'SaleCondition_Normal',
       u'SaleCondition_Partial'],
      dtype='object', length=217)

In [20]:
train = train.drop(['Id'], axis=1)
test = test.drop(['Id'], axis=1)

In [21]:
# import xgboost as xgb
X_train = train[[c for c in train if 'SalePrice' not in c and c in test]]
y_train = train['SalePrice']

X_test = test[[c for c in test if 'SalePrice' not in c]]
y_test = test['']

0       208500
1       181500
2       223500
3       140000
4       250000
5       143000
6       307000
7       200000
8       129900
9       118000
10      129500
11      345000
12      144000
13      279500
14      157000
15      132000
16      149000
17       90000
18      159000
19      139000
20      325300
21      139400
22      230000
23      129900
24      154000
25      256300
26      134800
27      306000
28      207500
29       68500
         ...  
1430    192140
1431    143750
1432     64500
1433    186500
1434    160000
1435    174000
1436    120500
1437    394617
1438    149700
1439    197000
1440    191000
1441    149300
1442    310000
1443    121000
1444    179600
1445    129000
1446    157900
1447    240000
1448    112000
1449     92000
1450    136000
1451    287090
1452    145000
1453     84500
1454    185000
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [22]:
preprocessing = Pipeline([
    ('impute', Imputer()),
    ('scale', StandardScaler())
])

X_train_sc = preprocessing.fit_transform(X_train)
X_test_sc = preprocessing.transform(test)

In [23]:
X_test_sc

array([[-0.87256276,  0.4519361 ,  0.11076257, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276,  0.49735662,  0.37584985, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.07337496,  0.17941296,  0.33205282, ..., -0.11785113,
         0.4676514 , -0.30599503],
       ..., 
       [-0.87256276,  4.08557799,  0.95042275, ..., -0.11785113,
        -2.13834494, -0.30599503],
       [ 0.66458604, -0.36563332, -0.00759964, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.07337496,  0.17941296, -0.08918038, ..., -0.11785113,
         0.4676514 , -0.30599503]])

In [24]:
model = XGBRegressor(
learning_rate = 0.4,
 n_estimators=1000,
 max_depth=8,
 objective= 'reg:linear',
 seed=27)
model.fit(X_train_sc, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.4, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=1)

In [25]:
y_pred = model.predict(X_test_sc)
preds = [round(value) for value in y_pred]

In [28]:
y_test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,0,1,0,0,0,1,0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,0,1,0,0,0,1,0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,0,1,0,0,0,1,0
5,60,75.0,10000,6,5,1993,1994,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
6,20,NaN,7980,6,7,1992,2007,0.0,935.0,0.0,...,0,0,0,0,1,0,0,0,1,0
7,60,63.0,8402,6,5,1998,1998,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
8,20,85.0,10176,7,5,1990,1990,0.0,637.0,0.0,...,0,0,0,0,1,0,0,0,1,0
9,20,70.0,8400,4,5,1970,1970,0.0,804.0,78.0,...,0,0,0,0,1,0,0,0,1,0


In [27]:
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test, preds)**0.5
RMSE

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
n = len(y_pred)
rmse = np.linalg.norm(y_pred - preds) / np.sqrt(n)
rmse

In [ ]:
# params =  {'learning_rate':0.5,
#  'max_depth':8,
#  'lambda':0.5,
#  'objective':'reg:linear',
#  'nthread':4,
#  'scale_pos_weight':1,
#  'eval_metric':'rmse',         
#  'seed':27,
#  'silent':1}

In [ ]:
# num_round = 1000
# model = xgb.train(params, dtrain, evals=evallist, num_boost_round=200, early_stopping_rounds=50)

In [ ]:
ypred = model.predict(dtest,ntree_limit=model.best_ntree_limit)

In [ ]:
xgb.plot_importance(booster=model).plot

In [ ]:
num_boost_round = 900

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'rmse'},
    early_stopping_rounds=100
)

cv_results

In [ ]:
output = pd.DataFrame({ 'Id' : ids, 'SalePrice': preds })
output.to_csv('housing-predictions.csv', index = False)
print(output.head())
len(output)